In [93]:
import sys
import os
import pandas as pd
current_path = os.getcwd()
parent_dir = os.path.join(current_path, "..")
print("Current working directory:", current_path)
print("Path to the parent directory:",parent_dir)
sys.path.append(parent_dir)
import simulator as sm
import numpy as np
import astropy.units as U
from casatasks import exportfits, simobserve, tclean, gaincal, applycal
from casatools import table
from casatools import simulator as casa_simulator
import random
import shutil

Current working directory: /home/astro/Documents/GitHub/ALMASim/experimental
Path to the parent directory: /home/astro/Documents/GitHub/ALMASim/experimental/..


Fixed Parameters

In [91]:
output_dir = current_path
plot_dir = current_path
project = 'test'
if not os.path.exists(os.path.join(output_dir, project)):
    os.mkdir(os.path.join(output_dir, project))
db = pd.read_csv(os.path.join(parent_dir, 'metadata', 'AGN_metadata.csv'))
metadata = db[['RA', 'Dec', 'Band', 'Ang.res.', 'FOV', 'Int.Time', 'Obs.date', 'Release date', 'PWV']]
n_px = 256
n_channels = 128
rest_frequency = 1420.4
metadata = metadata[metadata['Obs.date'] <= '2024-9-30']
idxs = [i for i in range(10)]
metadata = metadata.sample(n=len(idxs), replace=False)
n_channels = [n_channels for i in idxs]
n_pxs = [n_px for i in idxs]
total_times = [4500 for i in idxs]
antenna_configs = [sm.get_antenna_config_from_date(obs_date) for obs_date in metadata['Obs.date'].values]
antenna_ids, cycles = zip(*antenna_configs)
antenna_names = [os.path.join('cycle{}'.format(int(j)), 'alma.cycle{}.0.{}'.format(int(j), int(k))) for j, k in zip(cycles, antenna_ids)]

Observation Date: 2019-03-27
Computed Cycle 6 Antenna Config 2
Observation Date: 2022-10-15
Computed Cycle 9 Antenna Config 3
Observation Date: 2021-09-12
Computed Cycle 7 Antenna Config 9
Observation Date: 2023-12-26
Computed Cycle 10 Antenna Config 4
Observation Date: 2016-11-25
Computed Cycle 4 Antenna Config 3
Observation Date: 2022-08-16
Computed Cycle 8 Antenna Config 5
Observation Date: 2017-04-15
Computed Cycle 4 Antenna Config 3
Observation Date: 2022-10-26
Computed Cycle 9 Antenna Config 3
Observation Date: 2018-03-19
Computed Cycle 5 Antenna Config 4
Observation Date: 2022-06-03
Computed Cycle 8 Antenna Config 4


In [84]:
ras = metadata['RA'].values
decs = metadata['Dec'].values
bands = metadata['Band'].values
ang_res = metadata['Ang.res.'].values
fovs = metadata['FOV'].values
obs_dates = metadata['Obs.date'].values
release_dates = metadata['Release date'].values
pwvs = metadata['PWV'].values
int_times = metadata['Int.Time'].values

In [95]:
idx = 1

obs_date = obs_dates[idx]
antenna_name = antenna_names[idx]
band = int(bands[idx])
ra = ras[idx]
dec = decs[idx]
n_px = n_pxs[idx]
n_channel = n_channels[idx]
integration = int_times[idx]
total_time = total_times[idx]
pwv = pwvs[idx]
cycle = os.path.split(antenna_name)[0]
antenna_name = os.path.split(antenna_name)[1]
config_number = int(antenna_name.split('.')[-1])
# This function takes the spatial resolution 
spatial_resolution = sm.get_spatial_resolution(band, config_number)
central_freq = sm.get_band_central_freq(band)
# From the antenna configuration file, we get the maximum baseline, which is used to compute the beam size, and then cell_size
antennalist = os.path.join(parent_dir, "antenna_config", cycle, antenna_name + '.cfg')
max_baseline = sm.get_max_baseline_from_antenna_config(antennalist)
beam_size = sm.compute_beam_size_from_max_baseline(max_baseline, central_freq)
cell_size = beam_size / 5
# FoV only depends from the band (central frequency)
fov = sm.get_fov([band])[0]
print('Computed spatial resolution for band {} config {} and date {} is {} with respect to recorded {}'.format(band, config_number, obs_date, spatial_resolution, round(ang_res[idx], 2)))
print('Beam size:', beam_size)
print('Cell size:', cell_size)
print('Computed FoV from band {} is {} with respect to recorded {}'.format(band, round(fov, 2), round(fovs[idx], 2)))
print('Integration time:', integration)
print('Total time:', total_time)
print('\n')
#3683 of simulator function
n_sources = 4
bandwidth = 1280
inwidth = 10
serendipitous = True
run_tclean = False
fwhm_x = 1.5 * cell_size * np.random.rand() + cell_size
fwhm_y = 1.5 * cell_size  * np.random.rand() + cell_size
fwhm_z = 0.1 * bandwidth * np.random.rand() + inwidth
pa = np.random.randint(0, 360)
min_sep_spatial = 1.5 * cell_size
min_sep_frequency = 1.5 * inwidth
inbright = 0.001
filename = sm.generate_gaussian_skymodel(idx, current_path, n_sources,
                                                  n_px, n_channel, bandwidth, 
                                                  fwhm_x * U.arcsec, fwhm_y * U.arcsec, 
                                                  fwhm_z * U.MHz,
                                                  cell_size * U.arcsec,  
                                                  fov * U.arcsec, 
                                                  cell_size * U.arcsec, 
                                                  central_freq * U.GHz,
                                                  inwidth * U.GHz,
                                                  ra * U.deg, dec * U.deg,
                                                  pa, 
                                                  min_sep_spatial, 
                                                  min_sep_frequency,
                                                  rest_frequency, 
                                                  serendipitous,
                                                  current_path)

skymodel, sky_header = sm.load_fits(filename)
true_brightness = np.max(skymodel)
min_brightness = np.min(skymodel)
print('True Brightness', true_brightness, ' Jy/px')
print('Minimum Brightness', min_brightness, ' Jy/px')
if inbright != true_brightness:
    print('Adjusting Brightness')
    flattened_skymodel = np.ravel(skymodel)
    t_min = 0
    t_max = inbright
    skymodel_norm = (flattened_skymodel - np.min(flattened_skymodel)) / (np.max(flattened_skymodel) - np.min(flattened_skymodel)) * (t_max - t_min) + t_min
    skymodel = np.reshape(skymodel_norm, np.shape(skymodel))
    print('New Brightness', np.max(skymodel), ' Jy/px')
    sm.write_numpy_to_fits(skymodel, sky_header, filename)
simobserve(
    project=project, 
    skymodel=filename,
    inbright="{}Jy/pix".format(inbright),
    incell="{}arcsec".format(cell_size),
    indirection="J2000 19h30m00 -40d00m00",
    incenter='{}GHz'.format(central_freq),
    inwidth="{}MHz".format(inwidth),
    setpointings=True,
    integration="{}s".format(integration),
    obsmode="int",
    antennalist=antennalist,
    totaltime="{}s".format(total_time),
    thermalnoise="tsys-atm",
    user_pwv=pwv,
    graphics="none",
    verbose=False,
    overwrite=True)

# Adding atmosphere noise
#scale = random.uniform(0.3, 1)
scale = 0.5
print('Adding Atmospheric Noise using a scale factor of {} for thropospheric phase'.format(scale))
# scale is a multiplicative factor for the thropospheric phase 
# which is a delay in the propagation of radio waves in the atmosphere
# caused by the refractive index of the throphosphere
sm.simulate_atmospheric_noise(
    os.path.join(output_dir, project), 
    scale, 
    os.path.join(output_dir, project, "{}.{}.noisy.ms".format(project, antenna_name)), 
    antennalist)
gain_error_amplitude = random.gauss(0.001, 0.1)
sm.simulate_gain_errors(
    os.path.join(output_dir, project, "{}.{}.noisy.ms".format(project, antenna_name)),
    gain_error_amplitude
)

tclean(
    vis=os.path.join(project, "{}.{}.noisy.ms".format(project, antenna_name)),
    imagename=os.path.join(project, '{}.{}'.format(project, antenna_name)),
    imsize=[int(n_px), int(n_px)],
    cell="{}arcsec".format(cell_size),
    specmode="cube",
    niter=0,
    fastnoise=False,
    calcpsf=True,
    pbcor=True,
    pblimit=0.2,
    )

print('Saving Dirty and Clean Cubes')
exportfits(imagename=os.path.join(project, '{}.{}.image'.format(project, antenna_name)), 
       fitsimage=os.path.join(output_dir, "dirty_cube_" + str(idx) +".fits"), overwrite=True)
exportfits(imagename=os.path.join(project, '{}.{}.skymodel'.format(project, antenna_name)), 
        fitsimage=os.path.join(output_dir, "clean_cube_" + str(idx) +".fits"), overwrite=True)
sm.plotter(idx, output_dir, plot_dir, run_tclean, band, cycle, inbright, beam_size, cell_size, antenna_name, fwhm_z)
shutil.rmtree(project)
os.remove(filename)
os.remove(os.path.join(output_dir, "dirty_cube_" + str(idx) +".fits"))
os.remove(os.path.join(output_dir, "clean_cube_" + str(idx) +".fits"))
print('Done')

Computed spatial resolution for band 6 config 3 and date 2023-12-20 is 0.62 with respect to recorded 5.38
Beam size: 0.11792108103755336
Cell size: 0.023584216207510673
Computed FoV from band 6 is 25.15 with respect to recorded 45.14
Integration time: 108.864
Total time: 4500


2 1 6
Number of Pixels 256
Generating central source at position (127, 127, 64)
Generating central source and 4 serendipitous companions in a radius of 64 pixels in the x and y directions and 64 pixels in the z direction

Found 1st component
Found 2st component
Found 3st component
Found 4st component


  0%|          | 0/4 [00:00<?, ?it/s]

0:
Location: (128, 164, 21)
Size X: 1 Y: 1 Z: 5


 25%|██▌       | 1/4 [00:00<00:00,  3.42it/s]

1:
Location: (159, 86, 54)
Size X: 1 Y: 1 Z: 5


...

Skymodel saved to /home/astro/Documents/GitHub/ALMASim/experimental/skymodel_1.fits
True Brightness 1.8749809852613442  Jy/px
Minimum Brightness 0.0  Jy/px
Adjusting Brightness
New Brightness 0.001  Jy/px


....10....20....30....40....50....60....70....80....90....100%


Adding Atmospheric Noise using a scale factor of 0.5 for thropospheric phase


2024-03-26 16:41:22	WARN	task_tclean::SIImageStore::restore (file /source/casa6/casatools/src/code/synthesis/ImagerObjects/SIImageStore.cc, line 2265)	Restoring with an empty model image. Only residuals will be processed to form the output restored image.


Saving Dirty and Clean Cubes
